In [1]:
import json
data = []
with open("data/raw/bytecup.corpus.train.0.txt") as f:
    for line in f:
        data.append(json.loads(line))

In [2]:
import nltk

In [3]:
s_tokenizer = nltk.RegexpTokenizer('[\w\' ]+')
w_tokenizer = nltk.RegexpTokenizer('[\w\']+')

In [4]:
for cnt, d in enumerate(data):
    if cnt % 10000 == 0:
        print(cnt)
    d['sents'] = s_tokenizer.tokenize(d['content'])

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


In [5]:
def LCS(x, y):
    dp = [[0 for _ in y] for _ in x]
    for i in range(len(x)):
        for j in range(len(y)):
            if i > 0:
                dp[i][j] =  max(dp[i][j], dp[i - 1][j])
            if j > 0:
                dp[i][j] = max(dp[i][j], dp[i][j - 1])
            if x[i] == y[j]:
                if i > 0 and j > 0: dp[i][j] = max(dp[i][j], dp[i - 1][j - 1] + 1)
                else: dp[i][j] = 1   
    maxx = 0
    for i in range(len(x)):
        for j in range(len(y)):
            maxx = max(maxx, dp[i][j])
    return maxx

def rouge(x, y):
    global lx, ly
    lx += len(x)
    ly += len(y)
    lcs = LCS(x, y)
    if lcs == 0: return 0
    recall = lcs * 1.0 / len(y)
    precision = lcs * 1.0 / len(x)
    beta = precision / (recall + 1e-12)
    return (1 + beta * beta) * recall * precision / (recall + beta * beta * precision)

In [6]:
s, c = 0, 0
lx = 0
ly = 0
for d in data[3000:4000]:
    maxx = 0
    y = w_tokenizer.tokenize(d['title'])
    for sent in d['sents']:
        x = w_tokenizer.tokenize(sent)
        maxx = max(maxx, rouge(x, y))
    s += maxx
    c += 1

In [7]:
s / c

0.29116533835860325

In [38]:
import thread

def print_time( threadName, delay):
   count = 0
   while count < 5:
      time.sleep(delay)
      count += 1
      print "%s: %s" % ( threadName, time.ctime(time.time()) )

   thread.start_new_thread( print_time, ("Thread-1", 2, ) )
   thread.start_new_thread( print_time, ("Thread-2", 4, ) )

u'I\u2019m an angel investor'

In [71]:
from __future__ import print_function
with open('data/bytecup.corpus.validation_set.txt') as f:
    for line in f:
        obj = json.loads(line)
        content = obj['content']
        with open('data/result/%s.txt' % obj['id'], 'w') as fout:
            print(s_tokenizer.tokenize(content)[0].encode('utf8'), file=fout)